## 安裝套件&導入爬蟲的判決書

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv("crawling_law.csv",index_col=0) 
data.head()

,裁判字號,裁判日期,裁判案由,內文
0,\r\n 臺灣苗栗地方法院 110 年苗交簡字第 43...,\r\n 民國 110 年 07 月 01 日,\r\n 公共危險\r\n,\n\n\n\n\n臺灣苗栗地方法院刑事簡易判決 110年度苗交簡字第433號\r\n聲...
1,\r\n 臺灣花蓮地方法院 110 年花交簡字第 12...,\r\n 民國 110 年 07 月 01 日,\r\n 公共危險\r\n,\n\n\n\n\n臺灣花蓮地方法院刑事簡易判決 110年度花交簡字第129號\r\n聲...
2,\r\n 臺灣桃園地方法院 110 年桃交簡字第 16...,\r\n 民國 110 年 07 月 01 日,\r\n 公共危險\r\n,\n\n\n\n\n臺灣桃園地方法院刑事簡易判決 110年度桃交簡字第1616號\r\n...
3,\r\n 臺灣南投地方法院 110 年投交簡字第 26...,\r\n 民國 110 年 07 月 01 日,\r\n 公共危險\r\n,\n\n\n\n\n臺灣南投地方法院刑事簡易判決 110年度投交簡字第264號\r\n公...
4,\r\n 臺灣士林地方法院 110 年審交簡字第 10...,\r\n 民國 110 年 07 月 01 日,\r\n 公共危險\r\n,\n\n\n\n\n臺灣士林地方法院刑事簡易判決 110年度審交簡字第105號\r\n公...


## 資料處理

#### 處理欄位的雜質

In [ ]:
# 處理欄位的雜質
for i in range(len(data)):
    r = re.compile(r'\n|\r|\xa0|\u3000|\\u3000|\\u0020|\t|TRAN VAN NHI| |')
    data["裁判字號"][i] = r.sub('', data["裁判字號"][i])
    data["裁判日期"][i] = r.sub('', data["裁判日期"][i])
    data["裁判案由"][i] = r.sub('', data["裁判案由"][i])
    data["內文"][i] = r.sub('', data["內文"][i])

#### 將裁判日期分隔出月份

In [ ]:
# 分隔出月份
month = []
for i in range(len(data)):
    m = int(data["裁判日期"][i][6:8])
    month.append(m)
data["月分"] = month

#### 將裁判內文分隔出量型行其(以判刑N月為主)

In [ ]:
def get_key(target_key):
    digit_dict = {'壹月' : 1, '貳月' : 2, '參月' : 3, '肆月' : 4, '伍月' : 5, '陸月' : 6, 
                  '柒月' : 7, '捌月' : 8, '玖月' : 9, '拾月' : 10,'拾壹月':11, '拾貳月':12,
                  '一月' : 1, '二月' : 2, '三月' : 3, '四月' : 4, '五月' : 5, '六月' : 6,
                  '七月' : 7, '八月' : 8, '九月' : 9, '十月' : 10, '十一月':11, '十二月':12,
                  '1月' : 1, '2月' : 2, '3月' : 3, '4月' : 4, '5月' : 5, '6月' : 6,
                  '7月' : 7, '8月' : 8, '9月' : 9, '10月' : 10, '11月':11, '12月':12}
    for key, value in digit_dict.items():
         if target_key == key:
            return value
    return "None"

In [ ]:
sentence = re.findall("[一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿]*[月年]", re.findall("主文[^。]*。", data["內文"][200])[0])
get_key(sentence[0])

2

In [ ]:
# 分離出刑度
sentence_month = []
for i in range(len(data)):
    sentence = re.findall("[一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿]*[月年]", re.findall("主文[^。]*。", data["內文"][i])[0])
#     print(i, sentence)
    try:
        sentence_m = get_key(sentence[0])
        sentence_month.append(sentence_m)
    except:
        sentence_month.append("大於一年")
        pass
data["刑度月分"] = sentence_month

#### 分隔出酒測值

In [ ]:
# 分離出酒測值
value=[]
for i in data["內文"]:
    n= re.findall("[0-2]\.[0-9]+",i)
    temp = []
    for each in n:
        if each !='0.25':
            temp.append(each)
    try:
        value.append(list(set(temp))[0])
    except:
        value.append(0.0)
data["酒測值"]=value

### 輸出csv

In [ ]:
data.to_csv('law_data.csv')

In [ ]:
data = pd.read_csv("law_data.csv",index_col=0) 
data